In [1]:
import numpy as np
from scipy.integrate import solve_ivp

import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

# matplotlib imports
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation as animation

init_vprinting()

<img src="two_body_sketch.jpg" alt="Two Body Sketch" width=300>

We will attempt to predict the motion of two bodies in space, using the two-body problem. We will use the following assumptions:
* The two bodies are point masses
* The two bodies are not affected by any other bodies
* The two bodies are not affected by any other forces (e.g. friction, air resistance, etc.)
* The two bodies are not affected by relativity

To solve this, we could use Newton's laws of motions, but I would like to utilise Lagrangian formalism. This is because Lagrangian formalism is more general, and can be used to solve more complex problems. It is also more elegant, and is more intuitive to me.

We will use the following notation:

* $m_1$ and $m_2$ are the masses of the two bodies
* $\vec{r}_1$ and $\vec{r}_2$ are the positions of the two bodies
* $\dot{\vec{r}}_1$ and $\dot{\vec{r}}_2$ are the velocities of the two bodies
* $T$ is the kinetic energy of the system
* $V$ is the potential energy of the system
* $\mathcal{L}$ is the Lagrangian of the system

Our job is to find equations for the kinetic and potential energies of the system in terms of some general coordinates. These coordinates will be the vectors $\vec{r}_1$ and $\vec{r}_2$, the positions of our two bodies:

$$
\vec{r}_1 = \langle x_1, y_1, z_1 \rangle \quad \vec{r}_2 = \langle x_2, y_2, z_2 \rangle
$$

Afterwards, we can get the Lagrangian of the system through the following equation:

$$
\mathcal{L} = T - V
$$

To get the kinetic energy, we can use the following equation:

$$
T = \frac{1}{2}m_1v_1^2 + \frac{1}{2}m_2v_2^2
$$

We need to find a way to express the velocities in terms of the positions. We know that the velocities are the time derivatives of the positions, so we can express the kinetic energy as follows:

$$
T = \frac{1}{2}m_1|\dot{\vec{r}}_1|^2 + \frac{1}{2}m_2|\dot{\vec{r}}_2|^2
$$

The Lagrangian is then:

$$
\mathcal{L} = \frac{1}{2}m_1|\dot{\vec{r}}_1|^2 + \frac{1}{2}m_2|\dot{\vec{r}}_2|^2 - V
$$

The problem is that we don't know what $V$ is, so we will have to derive it ourselves. If $x_1$ and $x_2$ are the position vectors of the two bodies with respect to a fixed origin, then we can write the relative position vector as follows:

$$
\vec{r} = \vec{r}_1 - \vec{r}_2
$$

Now we want to get an equation for the barycentre of the system. The barycentre is the point where the two bodies would balance if they were connected by a rigid rod. To get an equation for the barycentre in terms of our positions and the masses, we can use the following equation:

$$
\vec{r}_{b} = \frac{m_1 \vec{r}_1}{m_1 + m_2} + \frac{m_2 \vec{r}_2}{m_1 + m_2}
$$

We can use our relative position vector to get an equation for the barycentre in terms of the relative position and the masses:

$$
\begin{align}
\vec{r}_{b} &= \frac{m_1 \vec{r}_1}{m_1 + m_2} + \frac{m_2 \vec{r}_2}{m_1 + m_2} \\
&= \frac{m_1(\vec{r}_2 + \vec{r})}{m_1 + m_2} + \frac{m_2\vec{r}_2}{m_1 + m_2} \\
&= \frac{m_1\vec{r}_2 + m_1\vec{r}}{m_1 + m_2} + \frac{m_2\vec{r}_2}{m_1 + m_2} \\
&= \frac{m_1\vec{r}_2 + m_1\vec{r} + m_2\vec{r}_2}{m_1 + m_2} \\
&= \frac{m_1\vec{r}_2 + m_2x_2 + m_1\vec{r}}{m_1 + m_2} \\
&= \frac{(m_1 + m_2)\vec{r}_2 + m_1\vec{r}}{m_1 + m_2} \\
&= \vec{r}_2 + \frac{m_1\vec{r}}{m_1 + m_2} \\
\end{align}
$$

In terms of $\vec{r}_1$, it is:

$$
\vec{r}_{b} = \vec{r}_1 - \frac{m_2\vec{r}}{m_1 + m_2}
$$

Therefore, the derivatives of the barycentre are:

$$
\begin{align}
\dot{\vec{r}}_{b} &= \dot{\vec{r}}_1 - \frac{m_2\dot{\vec{r}}}{m_1 + m_2} = \dot{\vec{r}}_2 + \frac{m_1\dot{\vec{r}}}{m_1 + m_2} \\
\ddot{\vec{r}}_{b} &= \ddot{\vec{r}}_1 - \frac{m_2\ddot{\vec{r}}}{m_1 + m_2} = \ddot{\vec{r}}_2 + \frac{m_1\ddot{\vec{r}}}{m_1 + m_2} \\
\end{align}
$$

If we define $\mu$ as the reduced mass of the system ($\mu = \frac{m_1m_2}{m_1 + m_2}$), then we can write the kinetic energy of the system as follows:

$$
\begin{align}
T &= \frac{1}{2}m_1|\dot{\vec{r}}_1|^2 + \frac{1}{2}m_2|\dot{\vec{r}}_2|^2 \\
&= \frac{1}{2}m_1\left(\dot{\vec{r}}_{b_{x}} + \frac{m_2\dot{\vec{r}}_x}{m_1 + m_2}\right)^2 + \frac{1}{2}m_1\left(\dot{\vec{r}}_{b_{y}} + \frac{m_2\dot{\vec{r}}_y}{m_1 + m_2}\right)^2 + \frac{1}{2}m_1\left(\dot{\vec{r}}_{b_{z}} + \frac{m_2\dot{\vec{r}}_z}{m_1 + m_2}\right)^2 \\
&+ \frac{1}{2}m_2\left(\dot{\vec{r}}_{b_{x}} - \frac{m_1\dot{\vec{r}}_x}{m_1 + m_2}\right)^2 + \frac{1}{2}m_2\left(\dot{\vec{r}}_{b_{y}} - \frac{m_1\dot{\vec{r}}_y}{m_1 + m_2}\right)^2 + \frac{1}{2}m_2\left(\dot{\vec{r}}_{b_{z}} - \frac{m_1\dot{\vec{r}}_z}{m_1 + m_2}\right)^2 \\
&= \frac{1}{2}m_1\left(\dot{\vec{r}}_{b_{x}}^2 + \frac{2m_2\dot{\vec{r}}_x\dot{\vec{r}}_{b_{x}}}{m_1 + m_2} + \frac{m_2^2\dot{\vec{r}}_x^2}{(m_1 + m_2)^2}\right) + \frac{1}{2}m_1\left(\dot{\vec{r}}_{b_{y}}^2 + \frac{2m_2\dot{\vec{r}}_y\dot{\vec{r}}_{b_{y}}}{m_1 + m_2} + \frac{m_2^2\dot{\vec{r}}_y^2}{(m_1 + m_2)^2}\right) \\
&+ \frac{1}{2}m_1\left(\dot{\vec{r}}_{b_{z}}^2 + \frac{2m_2\dot{\vec{r}}_z\dot{\vec{r}}_{b_{z}}}{m_1 + m_2} + \frac{m_2^2\dot{\vec{r}}_z^2}{(m_1 + m_2)^2}\right) + \frac{1}{2}m_2\left(\dot{\vec{r}}_{b_{x}}^2 - \frac{2m_1\dot{\vec{r}}_x\dot{\vec{r}}_{b_{x}}}{m_1 + m_2} + \frac{m_1^2\dot{\vec{r}}_x^2}{(m_1 + m_2)^2}\right) \\
&+ \frac{1}{2}m_2\left(\dot{\vec{r}}_{b_{y}}^2 - \frac{2m_1\dot{\vec{r}}_y\dot{\vec{r}}_{b_{y}}}{m_1 + m_2} + \frac{m_1^2\dot{\vec{r}}_y^2}{(m_1 + m_2)^2}\right) + \frac{1}{2}m_2\left(\dot{\vec{r}}_{b_{z}}^2 - \frac{2m_1\dot{\vec{r}}_z\dot{\vec{r}}_{b_{z}}}{m_1 + m_2} + \frac{m_1^2\dot{\vec{r}}_z^2}{(m_1 + m_2)^2}\right) \\
&= \frac{1}{2}m_1\dot{\vec{r}}_{b_{x}}^2 + \frac{1}{2}m_1\dot{\vec{r}}_{b_{y}}^2 + \frac{1}{2}m_1\dot{\vec{r}}_{b_{z}}^2 + \frac{1}{2}m_2\dot{\vec{r}}_{b_{x}}^2 + \frac{1}{2}m_2\dot{\vec{r}}_{b_{y}}^2 + \frac{1}{2}m_2\dot{\vec{r}}_{b_{z}}^2 \\
&+ \frac{m_1m_2\dot{\vec{r}}_x\dot{\vec{r}}_{b_{x}}}{m_1 + m_2} + \frac{m_1m_2\dot{\vec{r}}_y\dot{\vec{r}}_{b_{y}}}{m_1 + m_2} + \frac{m_1m_2\dot{\vec{r}}_z\dot{\vec{r}}_{b_{z}}}{m_1 + m_2} - \frac{m_1m_2\dot{\vec{r}}_x\dot{\vec{r}}_{b_{x}}}{m_1 + m_2} - \frac{m_1m_2\dot{\vec{r}}_y\dot{\vec{r}}_{b_{y}}}{m_1 + m_2} - \frac{m_1m_2\dot{\vec{r}}_z\dot{\vec{r}}_{b_{z}}}{m_1 + m_2} \\
&+ \frac{m_1^2m_2\dot{\vec{r}}_x^2}{2(m_1 + m_2)^2} + \frac{m_1^2m_2\dot{\vec{r}}_y^2}{2(m_1 + m_2)^2} + \frac{m_1^2m_2\dot{\vec{r}}_z^2}{2(m_1 + m_2)^2} + \frac{m_1m_2^2\dot{\vec{r}}_x^2}{2(m_1 + m_2)^2} + \frac{m_1m_2^2\dot{\vec{r}}_y^2}{2(m_1 + m_2)^2} + \frac{m_1m_2^2\dot{\vec{r}}_z^2}{2(m_1 + m_2)^2} \\
&= \frac{1}{2}\left(m_1 + m_2\right)\left(\dot{\vec{r}}_{b_{x}}^2 + \dot{\vec{r}}_{b_{y}}^2 + \dot{\vec{r}}_{b_{z}}^2\right) + \frac{m_1^2m_2}{2(m_1 + m_2)^2}\left(\dot{\vec{r}}_x^2 + \dot{\vec{r}}_y^2 + \dot{\vec{r}}_z^2\right) + \frac{m_1m_2^2}{2(m_1 + m_2)^2}\left(\dot{\vec{r}}_x^2 + \dot{\vec{r}}_y^2 + \dot{\vec{r}}_z^2\right) \\
&= \frac{1}{2}(m_1 + m_2)\left|\dot{\vec{r}}_b\right|^2 + \frac{m_1^2m_2\left|\dot{\vec{r}}\right|^2 + m_1m_2^2\left|\dot{\vec{r}}\right|^2}{2(m_1 + m_2)^2} \\
&= \frac{1}{2}(m_1 + m_2)\left|\dot{\vec{r}}_b\right|^2 + \frac{m_1m_2\left|\dot{\vec{r}}\right|^2}{2(m_1 + m_2)} \\
&= \frac{1}{2}(m_1 + m_2)\left|\dot{\vec{r}}_b\right|^2 + \frac{\mu\left|\dot{\vec{r}}\right|^2}{2} \\
&= \frac{1}{2}(m_1 + m_2)\dot{r}_b^2 + \frac{\mu\dot{r}^2}{2}



\end{align}
$$

Thus, the Lagrangian is:

$$
\begin{align}
\mathcal{L} &= T - V \\
&= \frac{1}{2}(m_1 + m_2)\dot{r}_b^2 + \frac{\mu\dot{r}^2}{2} - V
\end{align}
$$

Now, this problem has 6 degrees of freedom, but we can reduce it to 3 by using the fact that the space is homogeneous. This implies that the total linear momentum of the system is conserved, or that $\dot{\vec{r}}_{b}$ is constant - the velocity of the barycentre is constant. This could also be deduced by noticing that $\vec{r}_b$ is a cyclic coordinate (it does not appear in the Lagrangian), and thus the conjugate momentum $\vec{p}_b$ is conserved. Since $\vec{p}_b = m\dot{\vec{r}}_b$, the velocity of the barycentre is constant. With this, we may write the Lagrangian in the reference frame of the barycentre:

$$
\mathcal{L} = \frac{\mu\dot{r}^2}{2} - V \\
$$

We have thus reduced the problem to a single particle of mass $\mu$ moving in a potential $V$, a one-body problem. The degrees of freedom of the problem has been reduced from 6 to 3. We can consider another symmetry of the problem: the fact that the space is isotropic. This implies that the angular momentum, $\vec{L} = \vec{r} \times \vec{p}$, is conserved. We know that linear momentum is: $\vec p = m \vec v = m \dot{\vec{r}}$. We also can use our 3 degrees of freedom definitions of $\vec r_1$ and $\vec r_2$ (the equations above except with $\vec{r}_b$ set to 0). With these, we can get an equation for the angular momentum:

$$
\begin{align}
\vec{L} &= \vec{r} \times \vec{p} \\
&= \vec{r}_1 \times m_1\dot{\vec{r}}_1 + \vec{r}_2 \times m_2\dot{\vec{r}}_2 \\
&= m_1 \vec{r}_1 \times \dot{\vec{r}}_1 + m_2 \vec{r}_2 \times \dot{\vec{r}}_2 \\
&= m_1 \left(\frac{m_2}{m_1 + m_2}\vec{r}\right) \times \dot{\vec{r}}_1 + m_2 \left(-\frac{m_1}{m_1 + m_2}\vec{r}\right) \times \dot{\vec{r}}_2 \\
&= \frac{m_1m_2}{m_1 + m_2}\left(\vec{r} \times \dot{\vec{r}}_1 - \vec{r} \times \dot{\vec{r}}_2\right) \\
&= \mu\left(\vec{r} \times \dot{\vec{r}}_1 - \vec{r} \times \dot{\vec{r}}_2\right) \\
&= \mu\left(\vec{r} \times \left(\dot{\vec{r}}_1 - \dot{\vec{r}}_2\right)\right) \\ 
&= \mu\left(\vec{r} \times \dot{\vec{r}}\right) \\

\end{align}
$$

Let's say that our relative position vector $\vec{r}$ is in the $xy$-plane, $\vec{r} = \langle x, y, 0 \rangle$. Then, $\dot{\vec{r}} = \langle \dot{x}, \dot{y}, 0 \rangle$. The angular momentum is then:

$$
\begin{align}
\vec{L} &= \mu\left(\vec{r} \times \dot{\vec{r}}\right) \\
&= \mu\left(\langle x, y, 0 \rangle \times \langle \dot{x}, \dot{y}, 0 \rangle\right) \\
&= \mu\left(\left(y\cdot0 - 0\cdot\dot{y}\right)\hat{z} - \left(x\cdot0 - 0\cdot\dot{x}\right)\hat{y} + \left(x\cdot\dot{y} - y\cdot\dot{x}\right)\hat{z}\right) \\
&= \mu\left(\left(x\cdot\dot{y} - y\cdot\dot{x}\right)\hat{z}\right) \\
&= \langle 0, 0, \vec{L}_z \rangle \\
\end{align}
$$

We see that if we choose $\vec{r}$ to be in the $xy$-plane, then the angular momentum is in the $z$-direction. Therefore, the motion of the particle is constrained to the $xy$-plane as well. Thus, we may write the kinetic energy of the system with polar coordinates ($r$, $\phi$) with the origin at the barycentre. If we do this, then the y-coordinate of the particle is $r\sin\phi$, and the x-coordinate is $r\cos\phi$. The time derivatives of these are $\dot{y} = \dot{r}\sin\phi + r\cos\phi\dot{\phi}$ and $\dot{x} = \dot{r}\cos\phi - r\sin\phi\dot{\phi}$. Thus, the velocity is:

$$
v = \sqrt{\dot{x}^2 + \dot{y}^2} = \sqrt{\left(\dot{r}\cos\phi - r\sin\phi\dot{\phi}\right)^2 + \left(\dot{r}\sin\phi + r\cos\phi\dot{\phi}\right)^2} = \sqrt{\dot{r}^2 + r^2\dot{\phi}^2}
$$

With the kinetic energy formula, where $\mu$ replaces $m$, we may use polar coordinates now:

$$
T = \frac{\mu}{2}\left(\dot{r}^2 + r^2\dot{\phi}^2\right)
$$

Now, this only has 2 degrees of freedom. Let's rewrite the Lagrangian in terms of these coordinates:

$$
\mathcal{L} = \frac{\mu}{2}\left(\dot{r}^2 + r^2\dot{\phi}^2\right) - V
$$

We know that the Euler-Lagrange equations are:

$$
\frac{d}{dt}\left(\frac{\partial\mathcal{L}}{\partial\dot{q}_i}\right) - \frac{\partial\mathcal{L}}{\partial q_i} = 0
$$

We can see how $\phi$ is a cyclic coordinate, and thus the conjugate momentum $p_\phi$ is conserved. We can see how the partial derivative of $\mathcal{L}$ with respect to $\phi$ is 0. Thus, $\dot{\phi}$ is constant. Let's put our Lagrangian into the Euler-Lagrange equations:

$$
\begin{align}
\frac{d}{dt}\left(\frac{\partial\mathcal{L}}{\partial\dot{r}}\right) - \frac{\partial\mathcal{L}}{\partial r} &= 0 \\
\frac{d}{dt}\left(\mu\dot{r}\right) - \mu r\dot{\phi}^2 + \frac{\partial V}{\partial r} &= 0 \\
\mu\ddot{r} - \mu r\dot{\phi}^2 + \frac{\partial V}{\partial r} &= 0 \\
\mu\ddot{r} &= \mu r\dot{\phi}^2 - \frac{\partial V}{\partial r} \\
\ddot{r} &= r\dot{\phi}^2 - \frac{1}{\mu}\frac{\partial V}{\partial r} \\

\end{align}
$$

$$
\begin{align}
\frac{d}{dt}\left(\frac{\partial\mathcal{L}}{\partial\dot{\phi}}\right) - \frac{\partial\mathcal{L}}{\partial \phi} &= 0 \\
\frac{d}{dt}\left(\mu r^2\dot{\phi}\right) &= 0 \\
\end{align}
$$

Now, let's define a variable $\ell = \mu r^2\dot{\phi}$. If we solve for $\dot{\phi}$, we get $\dot{\phi} = \frac{\ell}{\mu r^2}$ and $\dot{\phi}^2 = \frac{\ell^2}{\mu^2 r^4}$. Let's substitute this into our equation for $\mu \ddot{r}$:

$$
\begin{align}
\mu\ddot{r} &= \mu r\dot{\phi}^2 - \frac{\partial V}{\partial r} \\
&= \mu r\frac{\ell^2}{\mu^2 r^4} - \frac{\partial V}{\partial r} \\
&= \frac{\ell^2}{\mu r^3} - \frac{\partial V}{\partial r} \\
\end{align}
$$

We could introduce an effective potential $V_{eff}$, where $V_{eff} = \frac{\ell^2}{2\mu r^2} + V$. Then, we can write:

$$
\mu\ddot{r} = -\frac{\partial V_{eff}}{\partial r}
$$